In [1]:
from pathlib import Path
import quakeio
import ssid as si
import OKID
import numpy as np

import pandas as pd

In [2]:
# channels = ["22", "24"]
# channels = ["23", "25"]
channels = dict( # PAINTER RIO DELL
    inputs  = [17, 3, 20],
    outputs = [ 9, 7 , 4]
)
# channels = dict(# Case 2
#     inputs  = [3,6,17],
#     outputs = [12, 14, 19]
# )
# channels = dict( # Case 5
#     inputs  = [1],
#     outputs = [16]
# )
# channels = dict(
#     inputs  = "2 7 25".split(),
#     outputs = "18 13 15 23 20".split()
# )

In [3]:
for file in Path("painter").glob("RioDell_P*.zip"):
    event = quakeio.read(file)
    try:
        inputs = [
            event.match("r", file_name=f".*{chan}.*").accel
            for chan in channels["inputs"]
        ]
        outpts = [
            event.match("r", file_name=f".*{chan}.*").accel
            for chan in channels["outputs"]
        ]
    except:
        print(f"failed {file.name}")
        
    else:
        dt = inputs[0]["time_step"]
        V = OKID.okid(inputs, outpts, dt=dt, kmax=500, mro=10, orm=4, verbose=True)
        print(si.IdentifiedSystem(dt, **V))
        # break

        # A,B,C,D = OKID.okid(inputs, outpts, dt=dt, kmax=500, mro=10, orm=4, verbose=True)
        # freqdmpSRIM, modeshapeSRIM, *_ = si.ComposeModes(dt, A, B, C, D)
        # Add validation
        # print(si.IdentifiedSystem(dt, A, B, C, D))
        # print(file, np.real(1/freqdmpSRIM[:,0]))

Error extracting modes: too many indices for array: array is 1-dimensional, but 2 were indexed


In [5]:
V["B"]

array([[-0.1592585 , -0.03648066,  0.30002533],
       [-0.3610846 , -0.2489186 , -0.27651246],
       [-0.02462627, -0.07882047, -0.06064352],
       [-0.09863061,  0.00874634,  0.23643018]])

In [13]:
pd.DataFrame(V["M"])

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0.346367,0.055495,-0.002365,-0.533396,-0.136568,-0.027011,1.448927,-0.105769,-0.008909,0.550272,...,-0.019947,0.311529,0.017744,-0.051129,0.067649,0.069027,-0.000885,-0.233905,-0.083483,0.057748
1,-0.002701,0.115101,-0.018536,0.011118,-0.215169,0.014922,0.066532,2.356402,0.043070,-0.010617,...,-0.020823,0.016470,0.621127,0.055977,-0.013417,0.040023,0.011951,-0.000698,-0.226978,-0.018137
2,-0.078413,0.107353,0.188322,0.181211,-0.077875,-0.266185,-0.018942,0.231403,1.932236,-0.256097,...,-0.027227,0.055350,-0.231221,0.477762,0.087240,0.158267,0.016408,-0.038107,0.065641,-0.236329


In [7]:
[y[0] for y in V["Y"][-13:]]

[array([ 1.69532602e-38, -5.96983665e-38, -9.26902802e-38]),
 array([ 4.49510895e-38,  4.61057772e-38, -6.49897388e-38]),
 array([ 6.90410705e-38,  1.46143917e-37, -3.31939343e-38]),
 array([8.75601736e-38, 2.33271451e-37, 2.91145529e-40]),
 array([9.92884772e-38, 3.01445722e-37, 3.29915088e-38]),
 array([1.03527793e-37, 3.46140385e-37, 6.25466515e-38]),
 array([1.00140549e-37, 3.64640054e-37, 8.68786564e-38]),
 array([8.95470136e-38, 3.56196499e-37, 1.04336882e-37]),
 array([7.26820086e-38, 3.22038218e-37, 1.13808278e-37]),
 array([5.09151817e-38, 2.65236003e-37, 1.14786080e-37]),
 array([2.59414741e-38, 1.90437590e-37, 1.07392842e-37]),
 array([-3.49575328e-40,  1.03493654e-37,  9.23571781e-38]),
 array([-2.60156997e-38,  1.10048387e-38,  7.09469438e-38])]

In [14]:
def hankel(Y, d, shift):
    from numpy.lib.stride_tricks import as_strided
    if shift:
        Y = np.roll(Y,1)
    return as_strided()
    return np.lib.stride_tricks.sliding_window_view(
            Y, (Y.shape[0], Y.shape[1]+1-d)
    ).reshape(d*Y.shape[0], -1)

In [ ]:
hankel(np.array(V["Y"]), 10, 0)

In [ ]:
np.array(V["dati"]).shape

In [ ]:
print(si.IdentifiedSystem(dt, -V["A"]))